# 🎨 GAN for Face Generation

Welcome to **Generative Adversarial Networks**! In this notebook, we'll create realistic face images from random noise using the power of adversarial training.

## What you'll learn:
- GAN architecture and adversarial training
- Generator and discriminator design
- DCGAN implementation
- Techniques for stable GAN training

Let's create new faces! 👤

In [ ]:
# Import libraries
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import os

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adam

plt.style.use('seaborn-v0_8')
np.random.seed(42)
tf.random.set_seed(42)

print(f"TensorFlow version: {tf.__version__}")
print(f"GPU Available: {len(tf.config.list_physical_devices('GPU')) > 0}")

# Set up directories
os.makedirs('generated_images', exist_ok=True)
os.makedirs('models', exist_ok=True)

In [ ]:
# Generate synthetic face dataset for demonstration
def create_synthetic_faces(num_samples=10000, img_size=64):
    """Create synthetic face-like images for training"""
    np.random.seed(42)
    
    # Create face-like patterns
    faces = []
    for i in range(num_samples):
        # Create a face-like structure
        face = np.zeros((img_size, img_size, 3))
        
        # Face oval
        center_x, center_y = img_size // 2, img_size // 2
        for x in range(img_size):
            for y in range(img_size):
                dist = ((x - center_x) ** 2 / (img_size // 3) ** 2 + 
                       (y - center_y) ** 2 / (img_size // 2.5) ** 2)
                if dist < 1:
                    face[x, y] = [0.8, 0.7, 0.6]  # Skin tone
        
        # Eyes
        eye_y = center_y - img_size // 6
        for eye_x in [center_x - img_size // 6, center_x + img_size // 6]:
            for x in range(eye_x - 3, eye_x + 4):
                for y in range(eye_y - 2, eye_y + 3):
                    if 0 <= x < img_size and 0 <= y < img_size:
                        face[x, y] = [0.1, 0.1, 0.1]  # Dark eyes
        
        # Nose
        nose_x, nose_y = center_x, center_y
        for x in range(nose_x - 1, nose_x + 2):
            for y in range(nose_y - 2, nose_y + 3):
                if 0 <= x < img_size and 0 <= y < img_size:
                    face[x, y] = [0.7, 0.6, 0.5]  # Nose shadow
        
        # Mouth
        mouth_y = center_y + img_size // 6
        for x in range(center_x - 4, center_x + 5):
            for y in range(mouth_y - 1, mouth_y + 2):
                if 0 <= x < img_size and 0 <= y < img_size:
                    face[x, y] = [0.6, 0.3, 0.3]  # Mouth
        
        # Add noise for variation
        noise = np.random.normal(0, 0.1, face.shape)
        face = np.clip(face + noise, 0, 1)
        
        faces.append(face)
    
    return np.array(faces)

# Create dataset
print("🎭 Creating synthetic face dataset...")
IMG_SIZE = 64
BATCH_SIZE = 64
NOISE_DIM = 100

# Generate synthetic faces
faces_data = create_synthetic_faces(5000, IMG_SIZE)
print(f"Dataset shape: {faces_data.shape}")

# Normalize to [-1, 1] for GAN training
faces_data = (faces_data - 0.5) * 2

# Create dataset
dataset = tf.data.Dataset.from_tensor_slices(faces_data)
dataset = dataset.shuffle(1000).batch(BATCH_SIZE, drop_remainder=True)

# Visualize sample faces
fig, axes = plt.subplots(2, 5, figsize=(15, 8))
for i, ax in enumerate(axes.flat):
    # Convert back to [0, 1] for display
    img = (faces_data[i] + 1) / 2
    ax.imshow(img)
    ax.set_title(f'Sample {i+1}')
    ax.axis('off')

plt.suptitle('🎭 Synthetic Face Dataset Samples', fontsize=16)
plt.tight_layout()
plt.show()

In [ ]:
# Build Generator
def build_generator(noise_dim=100, img_size=64):
    """Build DCGAN Generator"""
    model = models.Sequential([
        # Dense layer to start
        layers.Dense(8 * 8 * 256, input_shape=(noise_dim,)),
        layers.BatchNormalization(),
        layers.LeakyReLU(alpha=0.2),
        layers.Reshape((8, 8, 256)),
        
        # Upsample to 16x16
        layers.Conv2DTranspose(128, (4, 4), strides=(2, 2), padding='same'),
        layers.BatchNormalization(),
        layers.LeakyReLU(alpha=0.2),
        
        # Upsample to 32x32
        layers.Conv2DTranspose(64, (4, 4), strides=(2, 2), padding='same'),
        layers.BatchNormalization(),
        layers.LeakyReLU(alpha=0.2),
        
        # Upsample to 64x64
        layers.Conv2DTranspose(3, (4, 4), strides=(2, 2), padding='same'),
        layers.Tanh()  # Output in [-1, 1]
    ])
    
    return model

# Build Discriminator
def build_discriminator(img_size=64):
    """Build DCGAN Discriminator"""
    model = models.Sequential([
        # Input layer
        layers.Conv2D(64, (4, 4), strides=(2, 2), padding='same', 
                     input_shape=(img_size, img_size, 3)),
        layers.LeakyReLU(alpha=0.2),
        layers.Dropout(0.3),
        
        # Downsample to 16x16
        layers.Conv2D(128, (4, 4), strides=(2, 2), padding='same'),
        layers.BatchNormalization(),
        layers.LeakyReLU(alpha=0.2),
        layers.Dropout(0.3),
        
        # Downsample to 8x8
        layers.Conv2D(256, (4, 4), strides=(2, 2), padding='same'),
        layers.BatchNormalization(),
        layers.LeakyReLU(alpha=0.2),
        layers.Dropout(0.3),
        
        # Classification layer
        layers.Flatten(),
        layers.Dense(1, activation='sigmoid')
    ])
    
    return model

# Create models
generator = build_generator(NOISE_DIM, IMG_SIZE)
discriminator = build_discriminator(IMG_SIZE)

print("🎨 Generator Architecture:")
generator.summary()

print("\n🔍 Discriminator Architecture:")
discriminator.summary()

In [ ]:
# Define loss functions and optimizers
cross_entropy = tf.keras.losses.BinaryCrossentropy()

def discriminator_loss(real_output, fake_output):
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    total_loss = real_loss + fake_loss
    return total_loss

def generator_loss(fake_output):
    return cross_entropy(tf.ones_like(fake_output), fake_output)

# Optimizers
generator_optimizer = Adam(learning_rate=0.0002, beta_1=0.5)
discriminator_optimizer = Adam(learning_rate=0.0001, beta_1=0.5)

print("✅ Loss functions and optimizers defined!")

In [ ]:
# Training step
@tf.function
def train_step(images):
    noise = tf.random.normal([BATCH_SIZE, NOISE_DIM])
    
    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        generated_images = generator(noise, training=True)
        
        real_output = discriminator(images, training=True)
        fake_output = discriminator(generated_images, training=True)
        
        gen_loss = generator_loss(fake_output)
        disc_loss = discriminator_loss(real_output, fake_output)
    
    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)
    
    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))
    
    return gen_loss, disc_loss

# Function to generate and save images
def generate_and_save_images(model, epoch, test_input):
    predictions = model(test_input, training=False)
    
    fig = plt.figure(figsize=(10, 10))
    for i in range(16):
        plt.subplot(4, 4, i+1)
        # Convert from [-1, 1] to [0, 1]
        img = (predictions[i] + 1) / 2
        plt.imshow(img)
        plt.axis('off')
    
    plt.suptitle(f'Generated Images - Epoch {epoch}', fontsize=16)
    plt.tight_layout()
    plt.savefig(f'generated_images/image_at_epoch_{epoch:04d}.png')
    plt.show()

print("🚀 Training functions ready!")

In [ ]:
# Train the GAN
EPOCHS = 50
num_examples_to_generate = 16
seed = tf.random.normal([num_examples_to_generate, NOISE_DIM])

# Track losses
gen_losses = []
disc_losses = []

print("🎨 Starting GAN Training...")
print(f"Epochs: {EPOCHS}, Batch Size: {BATCH_SIZE}")

for epoch in range(EPOCHS):
    epoch_gen_loss = []
    epoch_disc_loss = []
    
    # Training loop
    for batch in tqdm(dataset, desc=f'Epoch {epoch+1}/{EPOCHS}'):
        gen_loss, disc_loss = train_step(batch)
        epoch_gen_loss.append(gen_loss)
        epoch_disc_loss.append(disc_loss)
    
    # Record average losses
    avg_gen_loss = tf.reduce_mean(epoch_gen_loss)
    avg_disc_loss = tf.reduce_mean(epoch_disc_loss)
    gen_losses.append(avg_gen_loss)
    disc_losses.append(avg_disc_loss)
    
    # Print progress
    print(f'Epoch {epoch+1}: Gen Loss: {avg_gen_loss:.4f}, Disc Loss: {avg_disc_loss:.4f}')
    
    # Generate images every 10 epochs
    if (epoch + 1) % 10 == 0:
        generate_and_save_images(generator, epoch + 1, seed)

print("\n🎉 Training completed!")

In [ ]:
# Visualize training progress
fig, axes = plt.subplots(1, 2, figsize=(15, 6))

# Loss curves
axes[0].plot(gen_losses, label='Generator Loss', alpha=0.8)
axes[0].plot(disc_losses, label='Discriminator Loss', alpha=0.8)
axes[0].set_title('📉 GAN Training Losses')
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Loss')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Final generated images
final_images = generator(seed, training=False)
axes[1].axis('off')
axes[1].set_title('🎨 Final Generated Faces')

plt.tight_layout()
plt.show()

# Show grid of final generated faces
fig, axes = plt.subplots(4, 4, figsize=(12, 12))
for i, ax in enumerate(axes.flat):
    img = (final_images[i] + 1) / 2  # Convert to [0, 1]
    ax.imshow(img)
    ax.axis('off')

plt.suptitle('🎭 Generated Faces - Final Results', fontsize=16)
plt.tight_layout()
plt.show()

print(f"\n📊 Training Summary:")
print(f"Final Generator Loss: {gen_losses[-1]:.4f}")
print(f"Final Discriminator Loss: {disc_losses[-1]:.4f}")
print(f"Total Epochs: {EPOCHS}")

In [ ]:
# Latent space interpolation
def interpolate_latent_space(generator, start_noise, end_noise, steps=10):
    """Interpolate between two points in latent space"""
    interpolated_images = []
    
    for i in range(steps):
        alpha = i / (steps - 1)
        interpolated_noise = (1 - alpha) * start_noise + alpha * end_noise
        generated_image = generator(interpolated_noise, training=False)
        interpolated_images.append(generated_image[0])
    
    return interpolated_images

# Create interpolation
start_noise = tf.random.normal([1, NOISE_DIM])
end_noise = tf.random.normal([1, NOISE_DIM])
interpolated = interpolate_latent_space(generator, start_noise, end_noise, 10)

# Visualize interpolation
fig, axes = plt.subplots(1, 10, figsize=(20, 4))
for i, (ax, img) in enumerate(zip(axes, interpolated)):
    display_img = (img + 1) / 2  # Convert to [0, 1]
    ax.imshow(display_img)
    ax.set_title(f'Step {i+1}')
    ax.axis('off')

plt.suptitle('🌈 Latent Space Interpolation', fontsize=16)
plt.tight_layout()
plt.show()

# Save models
generator.save('models/generator.h5')
discriminator.save('models/discriminator.h5')
print("\n💾 Models saved successfully!")

## 🎉 Congratulations!

You've successfully built and trained a GAN for face generation! Here's what you've accomplished:

✅ **GAN Architecture**: Built generator and discriminator networks  
✅ **Adversarial Training**: Implemented the min-max game  
✅ **DCGAN**: Used convolutional layers for image generation  
✅ **Face Generation**: Created realistic synthetic faces  
✅ **Latent Space**: Explored interpolation between generated images  

### 🚀 Next Steps:
1. Try different GAN variants (StyleGAN, Progressive GAN)
2. Experiment with conditional generation
3. Implement Wasserstein GAN for stability
4. Move on to **Project 07: Style Transfer Project**

Ready for artistic AI? Let's transfer styles! 🎨